# Voxelization
In this notebook we will voxelize the input volume and construct a lattice from it. The method is copied from spatial computing (https://github.com/shervinazadi/spatial_computing_workshops) using the topogenesis library.  


## 0. Initialization
Importing all necessary libraries and specifying the inputs

In [1]:
import os
import topogenesis as tg
import pyvista as pv
import trimesh as tm
import numpy as np
import pandas as pd
import copy
import digital_twinning as dt

In [2]:
vs = 3
unit = [vs, vs, vs]

mesh_path_building = os.path.relpath('../data/OBJ_files/building_envelope.obj')
mesh_path_context = os.path.relpath('../data/OBJ_files/final_BAG3D.obj')

## 1. Input Mesh

In [3]:
# load the mesh from file
envelope_mesh = tm.load(mesh_path_building)
context_mesh = tm.load(mesh_path_context)
# Check if the mesh is watertight

print(envelope_mesh.is_watertight)
print(context_mesh.is_watertight)

True
False


### visualize the meshes

In [4]:
# convert mesh to pv_mesh
# initiating the plotter
p = pv.Plotter(notebook=True)

# adding the meshes
p.add_mesh(dt.tri_to_pv(envelope_mesh), color='#abd8ff')
p.add_mesh(dt.tri_to_pv(context_mesh), color='#aaaaaa')

# plotting
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(84701.87724865344, 448134.6587486534, 327.4372486534429),
 (84384.44, 447817.2215, 10.0),
 (0.0, 0.0, 1.0)]

In [5]:
#start point array
envelope_mesh.vertices += np.array([0,0,vs/2])

## 2. Voxelize the Mesh

In [6]:
# initialize the base lattice
base_lattice = tg.lattice(envelope_mesh.bounds, unit=unit, default_value=1, dtype=int)

# check which voxel centroids is inside the mesh
interior_condition = envelope_mesh.contains(base_lattice.centroids)

# reshape the interior condition to the shape of the base_lattice
interior_array = interior_condition.reshape(base_lattice.shape)

# convert the interior array into a lattice
interior_lattice = tg.to_lattice(interior_array, base_lattice.minbound, base_lattice.unit)

### visualize the voxization lattice

In [7]:
# initiating the plotter
p = pv.Plotter(notebook=True)

# fast visualization of the lattice
interior_lattice.fast_vis(p)

# adding the context mesh: white
p.add_mesh(dt.tri_to_pv(context_mesh), color='#aaaaaa')

# plotting
p.show(use_ipyvtk=True)


# #saving and plotting
# png_path = os.path.relpath('../ful_lowres_lattice.png')
# p.show(screenshot= png_path)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(84702.22192807864, 448135.0034280786, 328.2819280786408),
 (84384.44, 447817.2215, 10.5),
 (0.0, 0.0, 1.0)]

## 3. Saving the lattice to CSV

In [8]:
# save the voxelized lattice to a CSV
lattice_dir = '../data/CSV_files/voxelized_envelope.csv'
dt.save_to_clean_csv(interior_lattice, lattice_dir)